# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [148]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [151]:
def read_nodes(gfile):
    """flag is a variable which is 0 if we are in first comment section and 1 otherwise"""
    flag=0
    """Initialising a list for storing the node values"""
    node=[]
    for line in gfile:
        if line[0]=='\n': 
            """This condition stops the loop before algorithm can begin reading edges"""
            break
        elif line[0]!='*': 
            """This condition allows reading the codename of tree nodes into the list"""
            node.append(line[:2])
            flag=1
    #print line
    return node
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    pass

Next implement a function to read in the edges from the input file.

In [152]:
def read_edges(gfile):
    """Initialize a list for storing edges"""
    edge_list=[]
    for line in gfile:
        """Since gfile start directly for the edge section, break away only when a comment in encountered"""
        if line[0]=='*':
            #print 'correct end'
            break
        
        i=0
        m=len(line)
        #print 'length=',m

        while(line[i]!=':'): 
            """each line has some character before : which should be ignored"""
            i=i+1
        #print line[i]

        while (i<m) and (line[i] != '\n'):
            i=i+1
            temp=[] 
            """Initialising a temporary list that stores the nodes that interact with each other"""
            #print 'started a new set'
            while (i<m) and (line[i]!=';' and line[i] != '\n') :
                #print 'started a new node'
                #print line[i]
                start=i 
                """Index for starting the name of the node"""
                while (i<m) and (line[i] !=',' and line[i]!=';' and line[i] != '\n'):
                    #print line[i]
                    i=i+1
                end=i 
                """Index for the end of the node"""
                if line[i]==',':
                    i=i+1
                #print 'ended the node'
                temp.append(line[start:end]) 
                """Adding the nodes encountered into the temporary array"""
                #print 'added a node to the list'

                #print temp
            #print 'ended a set'    
            n=len(temp)
            #print n
            k=0
            for l in range(n):
                #print 'we are here:1'
                for j in range(l+1,n): 
                    """This loop adds all the edges to the edge list"""
                    #print 'we are here :2'
                    edge=(temp[l],temp[j])
                    edge_list.append(edge)
                    #print 'edge list=', edge_list
                    k=k+1
                    #print k
            #print edge_list            
    
    return edge_list
                    


        
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    pass

The following code should now correctly create the graph.

In [153]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [154]:
def Search_subroutine(graph,u,tree): 
    """This subroutine return connected component of the given node"""
    graph.node[u]['explored']=1
    for v in sorted(graph.neighbors(u)):
        if graph.node[v]['explored']==0:
            tree.append(v)
            Search_subroutine(graph,v,tree)
            
def Search(graph, root):


    for u in sorted(graph.nodes()):
         
        graph.node[u]['explored']=0 
        """An attribute that tell if the node has been visited by dfs or not"""
    tree=[root]       
    """To Record the nodes of the tree as they appear in dfs"""
    Search_subroutine(graph,root,tree)
    return tree
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    pass   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [155]:
ulysses = Search(G, 'OD')


Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [156]:
def connected_components(graph):
    for u in graph.nodes():
        graph.node[u]['explored']=0 
        """initialising that all nodes are unvisited in the beginning"""
    forest=[] 
    """Initialising a list of all the connected components"""
    for v in sorted(graph.nodes()):
        if graph.node[v]['explored']==0: 
            """Running dfs on the smallest unvisited node"""
            tree=[v] 
            Search_subroutine(graph,v,tree)
            forest.append(tree)
    return forest
    

We will check correctness of your code by verifying that your output list is identical to our solution.

In [157]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [158]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
